In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import List, Tuple
from pathlib import Path

# repository root: prefer file-based parent when available, else two levels up
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    ROOT = Path().resolve().parents[1]


In [21]:
# config

# can hardcode filename...
# default to the known file in data/ryan_match_data
DEFAULT_CSV_FILENAME = "data/ryan_match_data/matches_2025-11-22_02-03-56.csv" 

# col names
CHAMP_COLS = [
    "red_top","red_jg","red_mid","red_adc","red_sup",
    "blue_top","blue_jg","blue_mid","blue_adc","blue_sup"
]

WINNER_COL = "winner"    
MATCH_ID_COL = "match_id" 

OUTPUT_DIR = str(Path(ROOT) / "src" / "ryan" / "encoded_outputs")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Output directory:", OUTPUT_DIR)


Output directory: C:\Users\Laser\cs171-lol-draft-analysis\src\ryan\encoded_outputs


In [22]:
# choose input

csv_path_input = input(f"Enter path to your CSV file (or press Enter to use '{DEFAULT_CSV_FILENAME}'): ").strip()
if csv_path_input == "":
    csv_filename = DEFAULT_CSV_FILENAME
else:
    csv_filename = csv_path_input

possible_paths = [
    Path(csv_filename),
    Path(ROOT) / csv_filename,
    Path(ROOT) / "data" / "ryan_match_data" / csv_filename,
    Path(ROOT) / "unprocessed_data" / csv_filename,
    Path(ROOT) / "processed_splits" / csv_filename,
]

csv_path = None
for p in possible_paths:
    if p.exists():
        csv_path = str(p)
        break

if csv_path is None:
    raise FileNotFoundError(f"File not found in candidates: {possible_paths}")

print("Using CSV path:", csv_path)

Using CSV path: C:\Users\Laser\cs171-lol-draft-analysis\data\ryan_match_data\matches_2025-11-22_02-03-56.csv


In [23]:
# load data

df = pd.read_csv(csv_path)
print("Loaded DataFrame with shape:", df.shape)
df.head()
print(df.info())

Loaded DataFrame with shape: (107700, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107700 entries, 0 to 107699
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   red_top   81792 non-null   object
 1   red_jg    81801 non-null   object
 2   red_mid   81807 non-null   object
 3   red_adc   81799 non-null   object
 4   red_sup   81799 non-null   object
 5   blue_top  81804 non-null   object
 6   blue_jg   81800 non-null   object
 7   blue_mid  81791 non-null   object
 8   blue_adc  81806 non-null   object
 9   blue_sup  81805 non-null   object
 10  winner    107700 non-null  int64 
 11  match_id  107700 non-null  object
dtypes: int64(1), object(11)
memory usage: 9.9+ MB
None


In [24]:
# clean data

# strip whitespace
def strip_whitespace(df: pd.DataFrame) -> pd.DataFrame:
    """Strip leading/trailing whitespace from all object (string) columns."""
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()
    return df

# drop rows with invalid or incomplete data
def drop_invalid_champ_rows(
    df: pd.DataFrame,
    champ_cols: List[str]
) -> pd.DataFrame:
    """
    Remove rows with NaN or empty strings in any champion slot.
    """
    # drop NaNs in champ cols
    df_clean = df.dropna(subset=champ_cols)

    # drop rows where any champ col is "" or some obvious placeholder
    mask_nonempty = (df_clean[champ_cols] != "").all(axis=1)
    df_clean = df_clean[mask_nonempty].copy()

    return df_clean

# ensures winner column is numeric 0/1
def normalize_winner_column(df: pd.DataFrame, winner_col: str) -> pd.DataFrame:
    """
    Optional: Normalize winner column to 0/1 if needed.
    Assumes:
      - 1 = blue win, 0 = red win
    Adjust mapping if your encoding is different.
    """
    if winner_col not in df.columns:
        print(f"Winner column '{winner_col}' not found, skipping normalization.")
        return df
    
    # Try to coerce to numeric; if it fails, you can customize this
    df[winner_col] = pd.to_numeric(df[winner_col], errors="ignore")

    # If it's already 0/1, nothing to do. If it's strings like 'blue'/'red', map them:
    if df[winner_col].dtype == object:
        unique_vals = df[winner_col].unique()
        print("Winner column is non-numeric, unique values:", unique_vals)
        # Example mapping (customize if needed):
        # df[winner_col] = df[winner_col].map({"red": 0, "blue": 1})
    else:
        print("Winner column dtype after normalization:", df[winner_col].dtype)

    return df


df = strip_whitespace(df)
df = drop_invalid_champ_rows(df, CHAMP_COLS)
df = normalize_winner_column(df, WINNER_COL)

print("After cleaning, shape:", df.shape)
df.head()


Winner column dtype after normalization: int64
After cleaning, shape: (107700, 12)


C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\481216766.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[winner_col] = pd.to_numeric(df[winner_col], errors="ignore")


,red_top,red_jg,red_mid,red_adc,red_sup,blue_top,blue_jg,blue_mid,blue_adc,blue_sup,winner,match_id
0,Fiora,Qiyana,Ambessa,Mel,Ashe,Aatrox,Riven,Viktor,Xerath,Janna,1,NA1_5418233073
1,Fiora,Ivern,Sylas,MissFortune,Leona,Neeko,Graves,Riven,Ziggs,Morgana,1,NA1_5418227132
2,Yorick,Malphite,Pantheon,Corki,MissFortune,Jayce,Elise,Ambessa,Ezreal,Nunu,0,NA1_5418213545
3,Vayne,Diana,Akshan,Varus,Poppy,KSante,Ambessa,Viktor,Riven,Neeko,0,NA1_5418192529
4,Renekton,Fizz,Qiyana,Smolder,Rakan,Kayle,Ambessa,Yasuo,Xayah,Thresh,1,NA1_5418164436


In [25]:
# one-hot encoding with position awareness

def position_one_hot_encode(
    df: pd.DataFrame,
    champ_cols: List[str]
) -> pd.DataFrame:
    """
    One-hot encode each champion column separately.
    Converts dummy columns to uint8 to save memory.
    """
    df_encoded = pd.get_dummies(df, columns=champ_cols, prefix=champ_cols)

    # convert all dummy columns (which are bool/uint) to uint8
    dummy_cols = [c for c in df_encoded.columns if any(c.startswith(col + "_") for col in champ_cols)]
    df_encoded[dummy_cols] = df_encoded[dummy_cols].astype("uint8")

    return df_encoded


df_pos = position_one_hot_encode(df, CHAMP_COLS)

print("Position-encoded shape:", df_pos.shape)
df_pos.head()
# %% 
# 4a. (No intermediate save) Position-encoded dataframe is kept in-memory
pos_path = None
print("Position-encoded data ready in memory (not saved to CSV).")


Position-encoded shape: (107700, 1644)
Position-encoded data ready in memory (not saved to CSV).


In [26]:
# champion presence encoding (side-based)

def get_all_champions(df: pd.DataFrame, champ_cols: List[str]) -> List[str]:
    """
    Get sorted unique champion names from the 10 champ columns.
    """
    values = df[champ_cols].values.ravel()
    champs = pd.unique(values)
    champs = [c for c in champs if isinstance(c, str) and c != ""]
    champs = sorted(champs)
    return champs


def add_side_presence_features(
    df_original: pd.DataFrame,
    df_target: pd.DataFrame,
    champ_cols: List[str]
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Add red_has_X / blue_has_X columns to df_target based on df_original champ columns.
    Returns (df_with_presence, list_of_new_columns)
    """
    red_cols = [c for c in champ_cols if c.startswith("red_")]
    blue_cols = [c for c in champ_cols if c.startswith("blue_")]

    all_champs = get_all_champions(df_original, champ_cols)
    new_cols = []

    for champ in all_champs:
        red_col = f"red_has_{champ}"
        blue_col = f"blue_has_{champ}"

        df_target[red_col] = (
            df_original[red_cols].eq(champ).any(axis=1).astype("uint8")
        )
        df_target[blue_col] = (
            df_original[blue_cols].eq(champ).any(axis=1).astype("uint8")
        )

        new_cols.extend([red_col, blue_col])

    return df_target, new_cols


df_hybrid, presence_cols = add_side_presence_features(
    df_original=df,
    df_target=df_pos.copy(),
    champ_cols=CHAMP_COLS
)

print("Hybrid-encoded shape:", df_hybrid.shape)
df_hybrid.head()



C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_target[blue_col] = (
C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_target[red_col] = (
C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

Hybrid-encoded shape: (107700, 1990)


C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_target[blue_col] = (
C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_target[red_col] = (
C:\Users\Laser\AppData\Local\Temp\ipykernel_128780\877198075.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

,winner,match_id,red_top_Aatrox,red_top_Ahri,red_top_Akali,red_top_Akshan,red_top_Alistar,red_top_Ambessa,red_top_Amumu,red_top_Anivia,...,red_has_Ziggs,blue_has_Ziggs,red_has_Zilean,blue_has_Zilean,red_has_Zoe,blue_has_Zoe,red_has_Zyra,blue_has_Zyra,red_has_nan,blue_has_nan
0,1,NA1_5418233073,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,NA1_5418227132,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,NA1_5418213545,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,NA1_5418192529,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,NA1_5418164436,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# build final feature set (keep winner, match_id, etc.)

keep_cols = [WINNER_COL, MATCH_ID_COL]
keep_cols = [c for c in keep_cols if c in df_hybrid.columns]

final_df = df_hybrid.copy()

print("Final DataFrame shape:", final_df.shape)
final_df.head()

Final DataFrame shape: (107700, 1990)


,winner,match_id,red_top_Aatrox,red_top_Ahri,red_top_Akali,red_top_Akshan,red_top_Alistar,red_top_Ambessa,red_top_Amumu,red_top_Anivia,...,red_has_Ziggs,blue_has_Ziggs,red_has_Zilean,blue_has_Zilean,red_has_Zoe,blue_has_Zoe,red_has_Zyra,blue_has_Zyra,red_has_nan,blue_has_nan
0,1,NA1_5418233073,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,NA1_5418227132,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,NA1_5418213545,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,NA1_5418192529,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,NA1_5418164436,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# separate features and target, split data
X = df_hybrid.drop(columns=["winner"])   # change to your label column
y = df_hybrid["winner"]

# train vs temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,          # 70% train
    random_state=42,
    stratify=y
)

# validation vs test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,          # 15% val, 15% test
    random_state=42,
    stratify=y_temp
)

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

Train: (75390, 1989)
Val:   (16155, 1989)
Test:  (16155, 1989)


In [29]:
# save splits
out_dir = Path(OUTPUT_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

X_train.to_csv(str(out_dir / "X_train.csv"), index=False)
X_val.to_csv(str(out_dir / "X_val.csv"), index=False)
X_test.to_csv(str(out_dir / "X_test.csv"), index=False)

y_train.to_frame().to_csv(str(out_dir / "y_train.csv"), index=False)
y_val.to_frame().to_csv(str(out_dir / "y_val.csv"), index=False)
y_test.to_frame().to_csv(str(out_dir / "y_test.csv"), index=False)
